In [6]:

import torch
import numpy as np
from torch.utils.data import DataLoader
from dataset_process.dataset_processs import MyDataset
import torch.optim as optim
from time import time
from tqdm import tqdm
import os

from module.transformer import Transformer
from module.loss import Myloss
from utils.random_seed import setup_seed
from utils.visualization import result_visualization



setup_seed(30)  
reslut_figure_path = 'result_figure'  



test_interval = 5  
draw_key = 1  
file_name = "Shower"  

# 超参数设置
EPOCH = 10
BATCH_SIZE = 135
LR = 0.005
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # CPU or GPU
print(f'use device: {DEVICE}')

d_model = 64
d_hidden = 128
q = 12
v = 12
h = 12
N = 12
dropout = 0.2
pe =  True 
mask = True  
optimizer_name = 'Adagrad'


submodel =	{
  "humidity": [0,4,8],
  "temmperature": [1,5,9],
  "soundavg": [2,6,10],
  "soundpeak": [3,7,11],
  "humidity_temmperature": [0,1,4,5,8,9],
  "humidity_soundavg": [0,2,4,6,8,10],
  "humidity_soundpeak": [0,3,4,7,8,11],
  "temmperature_soundavg": [1,2,5,6,9,10],
  "temmperature_soundpeak": [1,3,5,7,9,11],
  "soundavg_soundpeak": [2,3,6,7,10,11],
  "humidity_temmperature_soundavg": [0,1,2,4,5,6,8,9,10],
  "humidity_temmperature_soundpeak": [0,1,3,4,5,7,8,9,11],
  "humidity_soundavg_soundpeak": [0,2,3,4,6,7,8,10,11],
  "temmperature_soundavg_soundpeak": [1,2,3,5,6,7,9,10,11],
  "humidity_temmperature_soundavg_soundpeak": [0,1,2,3,4,5,6,7,8,9,10,11],
}

model = "humidity_temmperature_soundavg_soundpeak"



index = submodel[model]
file_name = file_name + model

train_dataset = MyDataset(index, 'train')
test_dataset = MyDataset(index, 'test')

print(train_dataset.train_dataset.shape)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

DATA_LEN = train_dataset.train_len  
d_input = train_dataset.input_len  
d_channel = train_dataset.channel_len  
d_output = train_dataset.output_len  


print('data structure: [lines, timesteps, features]')
print(f'train data size: [{DATA_LEN, d_input, d_channel}]')
print(f'mytest data size: [{train_dataset.test_len, d_input, d_channel}]')
print(f'Number of classes: {d_output}')


net = Transformer(d_model=d_model, d_input=d_input, d_channel=d_channel, d_output=d_output, d_hidden=d_hidden,
                  q=q, v=v, h=h, N=N, dropout=dropout, pe=pe, mask=mask, device=DEVICE).to(DEVICE)

loss_function = Myloss()
if optimizer_name == 'Adagrad':
    optimizer = optim.Adagrad(net.parameters(), lr=LR)
elif optimizer_name == 'Adam':
    optimizer = optim.Adam(net.parameters(), lr=LR)


correct_on_train = []
correct_on_test = []

loss_list = []
time_cost = 0
print(f'Number of params: {sum(p.numel() for p in net.parameters())}')
print(model)


def test(dataloader, flag='test_set'):
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            y_pre, _, _, _, _, _, _ = net(x, 'test')
            _, label_index = torch.max(y_pre.data, dim=-1)
            total += label_index.shape[0]
            correct += (label_index == y.long()).sum().item()
        if flag == 'test_set':
            correct_on_test.append(round((100 * correct / total), 2))
        elif flag == 'train_set':
            correct_on_train.append(round((100 * correct / total), 2))
        print(f'Accuracy on {flag}: %.2f %%' % (100 * correct / total))

        return round((100 * correct / total), 2)



def train():
    net.train()
    max_accuracy = 0
    pbar = tqdm(total=EPOCH)
    begin = time()
    for index in range(EPOCH):
        for i, (x, y) in enumerate(train_dataloader):
            optimizer.zero_grad()

            y_pre, _, _, _, _, _, _ = net(x.to(DEVICE), 'train')

            loss = loss_function(y_pre, y.to(DEVICE))

            #print(f'Epoch:{index + 1}:\t\tloss:{loss.item()}')
            loss_list.append(loss.item())

            loss.backward()

            optimizer.step()

        if ((index + 1) % test_interval) == 0:
            current_accuracy = test(test_dataloader)
            test(train_dataloader, 'train_set')
            print(f'Epoch:{index + 1}')
            print(f':{max(correct_on_test)}%\t :{max(correct_on_train)}%')

            if current_accuracy > max_accuracy:
                max_accuracy = current_accuracy
                torch.save(net, f'saved_model/{file_name} batch={BATCH_SIZE}.pkl')

        pbar.update()

    os.rename(f'saved_model/{file_name} batch={BATCH_SIZE}.pkl',
              f'saved_model/{file_name} {max_accuracy} batch={BATCH_SIZE}.pkl')

    end = time()
    time_cost = round((end - begin) / 60, 2)

    result_visualization(loss_list=loss_list, correct_on_test=correct_on_test, correct_on_train=correct_on_train,
                         test_interval=test_interval,
                         d_model=d_model, q=q, v=v, h=h, N=N, dropout=dropout, DATA_LEN=DATA_LEN, BATCH_SIZE=BATCH_SIZE,
                         time_cost=time_cost, EPOCH=EPOCH, draw_key=draw_key, reslut_figure_path=reslut_figure_path,
                         file_name=file_name,
                         optimizer_name=optimizer_name, LR=LR, pe=pe, mask=mask)

use device: cuda:0
torch.Size([24300, 25, 12])
data structure: [lines, timesteps, features]
train data size: [(24300, 25, 12)]
mytest data size: [(6345, 25, 12)]
Number of classes: 2
Number of params: 1312580
humidity_temmperature_soundavg_soundpeak


In [ ]:
train()

In [91]:
res = []
filess = os.listdir('saved_model')
save_model_path = ""
file_namee = file_name + " "
for f in filess:
    if file_namee in f:
        save_model_path = "saved_model\\" + f
print(save_model_path)      
net = torch.load(save_model_path, map_location=torch.device(DEVICE))
def test2(dataloader, flag='test_set'):
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            y_pre, _, _, _, _, _, _ = net(x, 'test')
            #print(torch.softmax(y_pre.data, -1))
            label_index = torch.softmax(y_pre.data, -1)
            #print(label_index)
            #print(label_index[1])
            res.append(label_index.cpu().numpy())
        #print(correct)
        #print(total)
    
test2(test_dataloader)
res = np.concatenate(res)
res = res[:,1]
np.save(file_name + ".npy", res)

saved_model\Showerhumidity 96.0 batch=135.pkl
